In [1]:
from cbench.commands import *
from cbench import state
from plumbum.machines.paramiko_machine import ParamikoMachine

In [2]:
list_instances()
load_state()

2015-12-31 15:10:13,622 INFO Start of 'load_state' args: () kwargs: {}


2015-12-31 15:10:13,829 INFO End of 'load_state'


In [3]:
create_instances(2, state.CLUSTER_INSTANCES)
create_instances(1, state.YCSB_INSTANCES, 'ycsb')

2015-12-30 17:25:58,083 INFO Waiting for startup of i-700eb1cc..


2015-12-30 17:26:28,557 INFO Go for Green! State: {'Code': 0, 'Name': 'pending'}


2015-12-30 17:26:28,559 INFO Created instance with id: i-700eb1cc private ip: 172.31.17.207 public ip: None


2015-12-30 17:26:29,611 INFO Waiting for startup of i-ee0db252..


2015-12-30 17:26:59,984 INFO Go for Green! State: {'Code': 0, 'Name': 'pending'}


2015-12-30 17:26:59,986 INFO Created instance with id: i-ee0db252 private ip: 172.31.27.129 public ip: None


2015-12-30 17:27:00,746 INFO Waiting for startup of i-7c0eb1c0..


2015-12-30 17:27:31,205 INFO Go for Green! State: {'Code': 0, 'Name': 'pending'}


2015-12-30 17:27:31,207 INFO Created instance with id: i-7c0eb1c0 private ip: 172.31.16.121 public ip: None


[]

In [10]:
print(state.CLUSTER_INSTANCES)
# Create additional instance
create_instances(1, state.CLUSTER_INSTANCES)

['i-ee0db252', 'i-700eb1cc']
2015-12-30 17:46:32,255 INFO Waiting for startup of i-5a0db2e6..

2015-12-30 17:47:17,806 INFO Go for Green! State: {'Code': 0, 'Name': 'pending'}


2015-12-30 17:47:17,809 INFO Created instance with id: i-5a0db2e6 private ip: 172.31.23.2 public ip: None


[]

In [23]:
from cbench import util
util.run_cassandra('i-5a0db2e6')

2015-12-30 18:41:28,836 INFO Checking instance 'i-5a0db2e6' state: {'Code': 16, 'Name': 'running'}


2015-12-30 18:41:28,837 INFO Connecting to 52.29.185.51


2015-12-30 18:41:28,964 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2015-12-30 18:41:29,255 INFO Authentication (publickey) successful!
2015-12-30 18:41:30,374 INFO [chan 1] Opened sftp connection (server version 3)

2015-12-30 18:41:30,995 INFO Command returned:


2015-12-30 18:41:30,997 INFO 038b2ead1c16dca34351eb2043547e1762521dfe9bdb614ed2d77d185ab320c1



2015-12-30 18:41:31,137 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2015-12-30 18:41:31,433 INFO Authentication (publickey) successful!
2015-12-30 18:41:32,379 INFO [chan 1] Opened sftp connection (server version 3)

2015-12-30 18:41:32,652 INFO Docker containers status:


2015-12-30 18:41:32,655 INFO CONTAINER ID        IMAGE               COMMAND                CREATED             STATUS              PORTS               NAMES
038b2ead1c16        cassandra:2.2       "/docker-entrypoint.   2 seconds ago       Up 1 seconds                            cassy               



In [8]:
# create the cluster
create_cluster(state.CLUSTER_INSTANCES)

2015-12-30 17:30:57,267 INFO Checking instance 'i-ee0db252' state: {'Code': 16, 'Name': 'running'}


2015-12-30 17:30:57,275 INFO Connecting to 52.29.134.49


2015-12-30 17:30:57,548 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2015-12-30 17:30:58,194 INFO Authentication (publickey) successful!
2015-12-30 17:30:59,334 INFO [chan 1] Opened sftp connection (server version 3)

2015-12-30 17:31:04,973 INFO Command returned:


2015-12-30 17:31:04,975 INFO 36d13656b00593b54ef0a83cbc3082492e1a1e11747df1b35c7427fb6fd41c8d



...

2015-12-30 17:31:09,639 INFO Command returned:


2015-12-30 17:31:09,642 INFO cd183f4bc6336ee3221a2fb3b61e4cd376dad1a206d48cc63e0378d9583ded56



2015-12-30 17:31:09,784 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2015-12-30 17:31:10,067 INFO Authentication (publickey) successful!
2015-12-30 17:31:10,950 INFO [chan 1] Opened sftp connection (server version 3)

2015-12-30 17:31:11,259 INFO Docker containers status:


2015-12-30 17:31:11,262 INFO CONTAINER ID        IMAGE               COMMAND                CREATED             STATUS              PORTS               NAMES
cd183f4bc633        cassandra:2.2       "/docker-entrypoint.   2 seconds ago       Up 1 seconds                            cassy               



In [9]:
prepare_benchmark(add_args=["-p", "recordcount=100000"])

2015-12-30 17:31:36,329 INFO Creating keyspace YCSB and table..


2015-12-30 17:31:36,482 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2015-12-30 17:31:36,775 INFO Authentication (publickey) successful!
2015-12-30 17:31:37,687 INFO [chan 1] Opened sftp connection (server version 3)

2015-12-30 17:31:39,898 INFO Result: 


2015-12-30 17:31:39,953 INFO Loading YCSB data for workload 'workloads/workloada'


2015-12-30 17:31:40,133 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2015-12-30 17:31:40,459 INFO Authentication (publickey) successful!
2015-12-30 17:33:57,552 INFO Result: YCSB Client 0.1
Command line: -db com.yahoo.ycsb.db.CassandraCQLClient -P workloads/workloada -p hosts=172.31.27.129,172.31.17.207 -p recordcount=100000 -load
Datacenter: datacenter1; Host: /172.31.27.129; Rack: rack1
Datacenter: datacenter1; Host: /172.31.17.207; Rack: rack1
[OVERALL], RunTime(ms), 115478.0
[OVERALL], Throughput(ops/sec), 865.9658116697552
[CLEANUP], Operations, 1.0
[CLEANUP], AverageLatency(us), 24136.0
[CLEANUP], MinLatency(us), 24128.0
[CLEANUP], MaxLatency(us), 24143.0
[CLEANUP], 95thPercentileLatency(us), 24143.0
[CLEANUP], 99thPercentileLatency(us), 24143.0
[INSERT], Operations, 100000.0
[INSERT], AverageLatency(us), 1141.93886
[INSERT], MinLatency(us), 472.0
[INSERT], MaxLatency(us), 305407.0
[INSERT], 95thPercentileLatency(us), 1941.0
[INSERT], 99thPercentileLatency(us), 5219.0
[IN

In [19]:
import _thread

bench = _thread.start_new_thread(start_benchmark, (4, ["-p", "operationcount=10000000"]))

2015-12-30 18:32:00,785 INFO Starting run '20151230171236' with workload 'workloads/workloada'


9316

In [25]:
#remove_cassandra_instance(state.CLUSTER_INSTANCES[-1])
from cbench.util import connect

with connect('i-5a0db2e6') as remote:
        cmd = remote["sudo"]
        ret = cmd("docker", "exec", "cassy", "nodetool", "decommission")

        log.info("Nodetool decomissioning returned: {0}".format(ret))

2015-12-30 18:48:32,147 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2015-12-30 18:48:32,472 INFO Authentication (publickey) successful!
2015-12-30 18:48:33,353 INFO [chan 1] Opened sftp connection (server version 3)

ProcessExecutionError: Command line: ['cd', '/home/ubuntu', '&&', '/usr/bin/sudo', 'docker', 'exec', 'cassy', 'nodetool', 'decommission']
Exit code: 1
Stderr:  | time="2015-12-30T17:48:37Z" level=fatal msg="Error response from daemon: Container cassy is not running" 

In [11]:
import os
os.getcwd()
os.makedirs("C:\\Users\\Jonas\\PycharmProjects\\cassandra-bench\\results\\20151227171214")

In [18]:
wait_for_finish()

2015-12-30 18:20:51,113 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2015-12-30 18:20:51,412 INFO Authentication (publickey) successful!
2015-12-30 18:20:52,316 INFO [chan 1] Opened sftp connection (server version 3)

2015-12-30 18:20:52,707 INFO No more YCSB running!


In [26]:
gather_results()

2015-12-30 18:54:05,030 INFO Gathering results into 'C:\Users\Jonas\PycharmProjects\cassandra-bench\results\20151230171236'


2015-12-30 18:54:05,360 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2015-12-30 18:54:05,678 INFO Authentication (publickey) successful!
2015-12-30 18:54:07,091 INFO [chan 1] Opened sftp connection (server version 3)

2015-12-30 18:54:07,641 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2015-12-30 18:54:07,982 INFO Authentication (publickey) successful!
2015-12-30 18:54:08,953 INFO [chan 1] Opened sftp connection (server version 3)

2015-12-30 18:54:10,328 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2015-12-30 18:54:10,724 INFO Authentication (publickey) successful!
2015-12-30 18:54:11,609 INFO [chan 1] Opened sftp connection (server version 3)

Unhandled exception in thread started by <function start_benchmark at 0x000002554CCC9C80>


In [27]:
terminate_all()

2015-12-30 18:54:38,941 WARNING Going to terminate instance 'i-ee0db252'


2015-12-30 18:54:39,413 WARNING Going to terminate instance 'i-700eb1cc'


2015-12-30 18:54:39,655 WARNING Going to terminate instance 'i-7c0eb1c0'
